# Lab 12: Grid Localization using Bayes Filter (Virtual Robot)

### <span style="color:rgb(0,150,0)">It is recommended that you close any heavy-duty applications running on your system while working on this lab.</span>

<hr>


In [ ]:
%load_ext autoreload
%autoreload 2

import traceback
from notebook_utils import *
from Traj import *
import asyncio
import pathlib
import os
from utils import load_config_params
from full_localization import Localization

# Setup Logger
LOG = get_logger('demo_notebook.log')

# Init GUI and Commander
gui = GET_GUI()
cmdr = gui.launcher.commander

gui.show()

In [ ]:
# Start the simulator
START_SIM()

# Start the plotter
START_PLOTTER()

In [ ]:
class RealRobot():
    """A class to interact with the real robot
    """

    def __init__(self, commander, ble):
        # Load world config
        self.world_config = os.path.join(
            str(pathlib.Path(os.path.abspath(__file__)).parent), "config", "world.yaml")
        
        self.config_params = load_config_params(self.world_config)
        
        # Commander to commuincate with the Plotter process
        # Used by the Localization module to plot odom and belief
        self.cmdr = commander

        # ArtemisBLEController to communicate with the Robot
        self.ble = ble
        
    def set_vel(self, v, w):
        """Set command velocity
        
        Keyword arguments:
            v -- Linear Velocity
            w -- Angular Velocity
        """
        raise NotImplementedError("set_vel is not implemented")

    def get_pose(self):
        """Get robot pose
        
        Returns:
            current_odom -- Odometry Pose (meters, meters, degrees)
        """
        raise NotImplementedError("get_pose is not implemented")

    def perform_observation_loop(self, rot_vel=120):
        """Perform the observation loop behavior, where the robot does a 360 degree turn 
        in place while collecting equidistant (in the angular space) sensor data, 
        starting with the first sensor reading taken at the robot's current heading. 
        The number of sensor readings depends on "observations_count" defined in world.yaml.
        
        Keyword arguments:
            rot_vel -- (Optional) Angular Velocity for loop (degrees/second)
        Returns:
            sensor_ranges   -- A column numpy array of the range values (meters)
            sensor_bearings -- A column numpy array of the bearings at which the sensor readings were taken (degrees)
                               These values are not really used in the Localization module, so you may return a empty array
        """
        raise NotImplementedError("perform_observation_loop is not implemented")


In [ ]:
# Initialize Robot to communicate with the virtual robot and plotter
robot = RealRobot(cmdr)

# Initialize mapper
# Requires a VirtualRobot object as a parameter
mapper = Mapper(robot)

# Initialize your BaseLocalization object
# Requires a RealRobot object and a Mapper object as parameters
loc = Localization(robot, mapper)

## Plot Map
cmdr.plot_map()

# Run the Bayes Filter
The cells below utilizes the functions declared above to run each iteration of the Bayes filter algorithm to localize the robot in the grid. <br>

In [ ]:
# Reset Plots
cmdr.reset_plotter()

# Init Uniform Belief
loc.init_grid_beliefs()

# Get Observation Data by executing a 360 degree rotation motion
loc.get_observation_data()

# Run Update Step
loc.update_step()
loc.plot_update_step_data(plot_data=True)

# Plot Odom and GT
current_odom, current_gt = robot.get_pose()
cmdr.plot_gt(current_gt[0], current_gt[1])
cmdr.plot_odom(current_odom[0], current_odom[1])

In [ ]:
# Initialize the Trajectory object
traj = Trajectory(loc)

# Run through each motion steps
for t in range(0, traj.total_time_steps):
    print("\n\n-----------------", t, "-----------------")
    
    prev_odom, current_odom, prev_gt, current_gt = traj.execute_time_step(t)
        
    # Prediction Step
    prediction_step(current_odom, prev_odom)
    
    # Plot the odometry and prior belief distribution in the plotter
    loc.plot_prediction_step_data(plot_data=True)
    
    # Get Observation Data by executing a 360 degree rotation motion
    # and store the range values in loc.obs_range_data
    loc.get_observation_data()
    
    # Update Step
    update_step()
    
    # Plot belief in the plotter
    loc.plot_update_step_data(plot_data=True)

# Uncomment the below line to wait for keyboard input between each iteration.
#   input("Press Enter to Continue")
        
    print("-------------------------------------")